In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
from datasets import load_dataset

In [3]:
import numpy as np

In [4]:
from typing import Dict, List

In [5]:
from tqdm.notebook import tqdm

In [6]:
sst2 = load_dataset("stanfordnlp/sst2")

In [7]:
sst2["train"]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 67349
})

In [8]:
sst2["validation"]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 872
})

In [9]:
sst2["test"]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 1821
})

In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

In [11]:
def tokenize(batch: Dict[str, List]):
    return tokenizer(batch["sentence"], truncation=True, max_length=512)

In [12]:
tokenized_sst2 = sst2.map(tokenize, batched=True)

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    return accuracy.compute(
        predictions=np.argmax(predictions, axis=1),
        references=labels,
    )

In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google/mobilebert-uncased",
    num_labels=2,
)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model

MobileBertForSequenceClassification(
  (mobilebert): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_fe

In [19]:
model = model.to("mps")
model.device

device(type='mps', index=0)

In [20]:
training_args = TrainingArguments(
    output_dir="tmp/mobilebert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sst2["train"],
    eval_dataset=tokenized_sst2["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.194800,0.279231,0.917431
2,0.157700,0.367442,0.909404
3,0.112100,0.348334,0.922018
4,0.103300,0.399022,0.916284
5,0.074600,0.477361,0.909404


TrainOutput(global_step=21050, training_loss=898.4853809164256, metrics={'train_runtime': 9077.9515, 'train_samples_per_second': 37.095, 'train_steps_per_second': 2.319, 'total_flos': 1452184699411620.0, 'train_loss': 898.4853809164256, 'epoch': 5.0})

In [19]:
del model